# Clasificador de emails de Thunderbird a .csv

## Objetivo
Leer los mensajes diarios de `yum update` de las máquinas, y en base a eso crear una lista en un archivo .csv, con un "ok" si no ha habido problemas con la actualización o una x en caso de haberlos o no recibir correo.

## Organización
El programa va a tener dos partes claramente diferenciadas:
1. Análisis de los mensajes
2. Escritura del resultado en un archivo .csv

Para más información, leer el README

## 0. Preparación

Para la tarea que tenemos entre manos necesitamos el módulo pyexcel. Pero hemos de definir los objetos correspondientes al problema entre manos, a saber: número de máquinas, lista con sus nombres (ordenada), lista inicializada con "x", y mensajes de no-error.

In [33]:
#Inputs (en línea de comandos en caso de script.py)
sysargv1 = "18"
sysargv2 = "08"

num_to_month = {"01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril", "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto", "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre" }

In [17]:
import csv
import pyexcel as pe

In [28]:
xlsx_file = "updates.ods"
sheet = pe.get_book(file_name=xlsx_file)

sheet["Enero"][2,0] = "quepasa"

print(sheet["Enero"][2,0])


quepasa


In [19]:
okkeys = ["No se han seleccionando paquetes para ser actualizados","No packages marked for update", "¡Listo!", "Complete!"] #el primer mensaje de error tiene una errata, sí.
with open("/home/goznalo/CCC/servernames.csv", encoding='utf-8-sig') as file1: #modificar con el archivo pertinente
    listservers =  [server.replace('\n', '').lower() for server in file1]
numservers = len(listservers)
okxlist = ['x'] * numservers


En la lista de máquinas, gran parte de ellas no se cuentan a la hora de revisar estos correos porque no los mandan por defecto. Esto puede deberse a que  estén de baja, que sean sondas o SAIs que no mandan mensajes, etc. Vamos a eliminarlos de la lista de "x"

In [20]:
with open("/home/goznalo/CCC/nomessages.csv", encoding='utf-8-sig') as file2: #modificar con el archivo pertinente
    nomsglist =  [False if not server.replace('\n', '') else True for server in file2]
nomsglist.append(False) #el valor de zeus,  vacío ya que no manda mensajes

okxlist =  [a * b for a, b in zip(okxlist,nomsglist)]

# 1. Análisis de los mensajes

Los mensajes se encuentran en un archivo (que Thunderbird considera un directorio de mensajes) dentro de un directorio local creado y usado por Thunderbird.

In [21]:
directorio = "/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUMtotal.sbd/"
archivo = sysargv1 + sysargv2 #modificar con el archivo pertinente, o "prueba1"

path = directorio + archivo

In [22]:
print(path)

/home/goznalo/.thunderbird/ceesvdh2.default/Mail/fluor.ccc.uam-1.es/YUMtotal.sbd/1208


Ahora abriremos el archivo (con codificación "iso-8859-1", como podemos ver al ejecutar en una shell `file -i archivo`), que contiene cada uno de los mensajes de esa carpeta, y los separaremos en elementos de una lista, para luego iterar sobre ellos. Para efectuar esta separación, hemos de darnos cuenta de que en cada archivo de mensajes del directorio, los mensajes empiezan con una línea como por ejemplo `From - Wed Jul 29 15:16:48 2020`. Por tanto usaremos como separador `From -`

In [23]:
with open(path, 'r', encoding='iso-8859-1') as file3:
    messages = file3.read().replace('\n', '')

messlist = messages.split("From -")[1:]

Ahora que tenemos los mensajes, hemos de obtener una lista con los remitentes de cada uno de ellos, a la vez que comprobamos si tienen errores o no.

In [24]:
errlist = []
for mess in messlist:
    sender = mess.split("<root@")[1].split(">Received:")[0]
    num = listservers.index(sender)
    for oks in okkeys:
        err = 0
        if oks in mess:
            okxlist[num] = "ok"
            break
        else:
            err = 1
    if err == 1:
        errlist.append(sender)

## 2. Escritura del csv y mostrar correos sospechosos

Ya tenemos la lista con los datos que buscábamos. Ahora lo que hemos de hacer es convertirla en un archivo .csv, en formato columna.

In [25]:
with open(f'/home/goznalo/CCC/yum-thunderbird/returns{sysargv1 + sysargv2}.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for val in okxlist:
        writer.writerow([val])

Voilà

In [27]:
####Sacar por pantalla los correos a revisar
errlist = list(sorted(dict.fromkeys(errlist)))
print("\033[1mCorreos a revisar:\033[0m")
for err in errlist:
    print("\t" + str(err))

Correos a revisar:
	backupx.ccc.uam.es
	cibeles2-008.ccc.uam.es
	xchem70.ccc.uam.es
